# New York City Swimming Pools

In [32]:
import sedona.db
import os

os.environ["AWS_SKIP_SIGNATURE"] = "true"
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

sd = sedona.db.connect()

In [33]:
df = sd.read_parquet(
    "/Users/matthewpowers/data/overture/base_water.parquet"
    # "s3://overturemaps-us-west-2/release/2025-08-20.0/theme=base/type=water/"
)

In [34]:
df.schema

SedonaSchema with 13 fields:
  id: utf8<Utf8View>
  geometry: geometry<WkbView(ogc:crs84)>
  bbox: struct<Struct(xmin Float32, xmax Float32, ymin Float32, ymax Float32)>
  version: int32<Int32>
  sources: list<List(Field { name: "element", data_type: Struct([Field { name: "property", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "dataset", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "license", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "record_id", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "update_time", data_type: Utf8, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "confidence", data_type: Float64, nullable: true, dict_id: 0, dict_is_ordered: false, metadata: {} }, Field { name: "between", data_type: List(Field { name: "element", da

In [35]:
df.to_view("base_water", True)

In [36]:
nyc = "POLYGON((-74.26 40.50, -74.26 40.92, -73.70 40.92, -73.70 40.50, -74.26 40.50))"
query = f"""
select * from base_water
where
 ST_Within(geometry, ST_SetSRID(ST_GeomFromText('{nyc}'), 4326))
 and source_tags['leisure'] = 'swimming_pool'
"""

In [37]:
res = sd.sql(query)

In [38]:
res.show()

┌──────────────────┬──────────────────┬─────────────────┬───┬──────────┬─────────┬─────────────────┐
│        id        ┆     geometry     ┆       bbox      ┆ … ┆ wikidata ┆ is_salt ┆ is_intermittent │
│       utf8       ┆     geometry     ┆      struct     ┆   ┆   utf8   ┆ boolean ┆     boolean     │
╞══════════════════╪══════════════════╪═════════════════╪═══╪══════════╪═════════╪═════════════════╡
│ ca11462b-05f4-3… ┆ POLYGON((-74.24… ┆ {xmin: -74.249… ┆ … ┆          ┆         ┆                 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 88a60910-bf80-3… ┆ POLYGON((-74.24… ┆ {xmin: -74.249… ┆ … ┆          ┆         ┆                 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 7245307f-f19c-3… ┆ POLYGON((-74.24… ┆ {xmin: -74.249… ┆ … ┆          ┆         ┆                 │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌

In [39]:
res.count()

48976

In [40]:
from lonboard import viz

In [41]:
viz(res)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

## todo

figure it out if I can figure out a way to make the dots bigger in the map

In [19]:
from lonboard import Map, ScatterplotLayer

res_pd = res.to_pandas()
res_points = res_pd.copy()
res_points['geometry'] = res_points.geometry.centroid

/var/folders/rg/q_yx8y8n14db7cnn4382r6kh0000gn/T/ipykernel_87274/3326610380.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  res_points['geometry'] = res_points.geometry.centroid


In [20]:
len(res_points)

48976

In [21]:
layer = ScatterplotLayer.from_geopandas(
    points_only,
    get_radius=100,  # Adjust this value to make dots bigger
)

map = Map(layers=[layer])
map

Map(custom_attribution='', layers=(ScatterplotLayer(get_radius=100.0, table=arro3.core.Table
+----------------…